In [ ]:
# exporting models by optimum-cli
!optimum-cli export onnx --model openai/whisper-tiny whisper-tiny-with-past/ --task automatic-speech-recognition-with-past --opset 13


# Export and save model to onnx

In [47]:
import os
import time
import shutil
from evaluate import load
from pathlib import Path
from datasets import load_dataset
from transformers import WhisperForConditionalGeneration, WhisperProcessor, AutoConfig
from optimum.onnxruntime import ORTModelForSpeechSeq2Seq, ORTQuantizer, AutoQuantizationConfig
from transformers import PretrainedConfig
import librosa
import numpy as np

In [48]:
# Export model to ONNX
def export_onnx(model_id, save_dir):
    model = ORTModelForSpeechSeq2Seq.from_pretrained(model_id, export=True)
    model_dir = model.model_save_dir
    # print(model.get_memory_footprint())
        
    shutil.move(model_dir, save_dir)
    print("Model exported to onnx and saved at location ", save_dir)

In [ ]:
# export whisper-small
export_onnx("openai/whisper-small","/home/carol/mp/quantize/small-whisper")

In [ ]:
#export whisper-tiny
export_onnx("openai/whisper-xxxxx","/home/carol/mp/quantize/xxxxxxxxx")

In [49]:
#export whisper-base
export_onnx("openai/whisper-base","/home/carol/mp/quantize/base")

Framework not specified. Using pt to export the model.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using the export variant default. Available variants are:
    - default: The default ONNX variant.
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 1

Model exported to onnx and saved at location  /home/carol/mp/quantize/base


# Dynamic Quantization - intel (symmetric weights and activations)

In [50]:
from pathlib import Path
def quantize_nor(model_dir, save_dir):
    
    # create list of onnx models from the directory
    onnx_models = list(Path(model_dir).glob("*.onnx"))

    # instantiate quantizer and set quantization configuration
    quantizers = [ORTQuantizer.from_pretrained(model_dir, file_name=onnx_model) for onnx_model in onnx_models]
    qconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False,nodes_to_exclude=['/conv1/Conv', '/conv2/Conv'],use_symmetric_activations=True,  
        use_symmetric_weights=True,  
        operators_to_quantize=None )

    for quantizer in quantizers:
        # Apply dynamic quantization and save the resulting model
        quantizer.quantize(save_dir=save_dir, quantization_config=qconfig)
        print("quantized ", quantizer)
    print("Quantization complete")
    

# Dynamic Quantization - arm64

In [51]:
from pathlib import Path
def quantize_arm(model_dir, save_dir):
    
    # create list of onnx models from the directory
    onnx_models = list(Path(model_dir).glob("*.onnx"))

    # instantiate quantizer and set quantization configuration
    quantizers = [ORTQuantizer.from_pretrained(model_dir, file_name=onnx_model) for onnx_model in onnx_models]
    qconfig = AutoQuantizationConfig.arm64(is_static=False, per_channel=False,nodes_to_exclude=['/conv1/Conv', '/conv2/Conv'],use_symmetric_activations=True,  
        use_symmetric_weights=True,  
        operators_to_quantize=None )

    for quantizer in quantizers:
        # Apply dynamic quantization and save the resulting model
        quantizer.quantize(save_dir=save_dir, quantization_config=qconfig)
        print("quantized ", quantizer)
    print("Quantization complete")

In [9]:
quantize_sym("/home/carol/mp/quantize/tiny","/home/carol/mp/quantize/tiny-sym")

Creating dynamic quantizer: QOperator (mode: IntegerOps, schema: u8/s8, channel-wise: False)
Quantizing model...
Saving quantized model at: /home/carol/mp/quantize/tiny-sym (external data format: False)
Configuration saved in /home/carol/mp/quantize/tiny-sym/ort_config.json
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 2167

quantized  <optimum.onnxruntime.quantization.ORTQuantizer object at 0x7ed63444d060>


Quantizing model...
Saving quantized model at: /home/carol/mp/quantize/tiny-sym (external data format: False)
Configuration saved in /home/carol/mp/quantize/tiny-sym/ort_config.json
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870,

quantized  <optimum.onnxruntime.quantization.ORTQuantizer object at 0x7ed503998a00>


Quantizing model...
Saving quantized model at: /home/carol/mp/quantize/tiny-sym (external data format: False)
Configuration saved in /home/carol/mp/quantize/tiny-sym/ort_config.json
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870,

quantized  <optimum.onnxruntime.quantization.ORTQuantizer object at 0x7ed500deb670>
Quantization complete


In [ ]:
d_quantize("/home/carol/mp/quantize/small-whisper","/home/carol/mp/quantize/q-small")

In [18]:
quantize_sym("/home/carol/mp/quantize/small","/home/carol/mp/quantize/small-sym")

Creating dynamic quantizer: QOperator (mode: IntegerOps, schema: u8/s8, channel-wise: False)
Quantizing model...
Saving quantized model at: /home/carol/mp/quantize/small-sym (external data format: False)
Configuration saved in /home/carol/mp/quantize/small-sym/ort_config.json
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21

quantized  <optimum.onnxruntime.quantization.ORTQuantizer object at 0x7ed4ffe90820>


Quantizing model...
Saving quantized model at: /home/carol/mp/quantize/small-sym (external data format: False)
Configuration saved in /home/carol/mp/quantize/small-sym/ort_config.json
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 4987

quantized  <optimum.onnxruntime.quantization.ORTQuantizer object at 0x7ed4ffe90790>


Quantizing model...
Saving quantized model at: /home/carol/mp/quantize/small-sym (external data format: False)
Configuration saved in /home/carol/mp/quantize/small-sym/ort_config.json
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 4987

quantized  <optimum.onnxruntime.quantization.ORTQuantizer object at 0x7ed4f694cfa0>
Quantization complete


In [22]:
quantize_asym("/home/carol/mp/quantize/small","/home/carol/mp/quantize/small-asym")

Creating dynamic quantizer: QOperator (mode: IntegerOps, schema: u8/s8, channel-wise: False)
Quantizing model...
Saving quantized model at: /home/carol/mp/quantize/small-asym (external data format: False)
Configuration saved in /home/carol/mp/quantize/small-asym/ort_config.json
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 

quantized  <optimum.onnxruntime.quantization.ORTQuantizer object at 0x7ed500218d00>


Quantizing model...
Saving quantized model at: /home/carol/mp/quantize/small-asym (external data format: False)
Configuration saved in /home/carol/mp/quantize/small-asym/ort_config.json
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49

quantized  <optimum.onnxruntime.quantization.ORTQuantizer object at 0x7ed500218ca0>


Quantizing model...
Saving quantized model at: /home/carol/mp/quantize/small-asym (external data format: False)
Configuration saved in /home/carol/mp/quantize/small-asym/ort_config.json
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49

quantized  <optimum.onnxruntime.quantization.ORTQuantizer object at 0x7ed4f694d5a0>
Quantization complete






# Inference on ONNX models



In [43]:
import os
import librosa
import time
from transformers import WhisperProcessor, AutoConfig
from optimum.onnxruntime import ORTModelForSpeechSeq2Seq

def run(model_name, audio):
    
    #define odel path
    model_path = os.path.join("/home/carol/mp/quantize", model_name)
    # Load the model and processor
    processor = WhisperProcessor.from_pretrained(model_name)
    model_config = AutoConfig.from_pretrained(model_name)
    sessions = ORTModelForSpeechSeq2Seq.load_model(
                os.path.join(model_path, 'encoder_model.onnx'),
                os.path.join(model_path, 'decoder_model.onnx'),
                os.path.join(model_path, 'decoder_with_past_model.onnx'))
    model = ORTModelForSpeechSeq2Seq(sessions[0], sessions[1], model_config, model_path, sessions[2])

    # Load the audio file
    audio_data, sample_rate = librosa.load(audio, sr=16000, mono=True)

    # Preprocess the audio
    input_features = processor(audio_data, sampling_rate=sample_rate, return_tensors="pt").input_features
    forced_decoder_ids = processor.get_decoder_prompt_ids(language="english",task="translate")
    
    # Measure the time taken for inference
    start_time = time.time()
    predicted_ids = model.generate(input_features,forced_decoder_ids=forced_decoder_ids)[0]
    # Generate transcription
    transcription = processor.decode(predicted_ids, skip_special_tokens=True)
    inference_time = time.time() - start_time

     # model size
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(model_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    size = total_size / (1024 * 1024)  # Convert to MB

    print()
    print("Model name = ",model_name)
    print()
    print(transcription)
    print()
    print("Inference Time = ",inference_time)
    print("Model size = ",size) 
    

In [44]:
run("small","sample.wav")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



Model name =  small

 Harvard List number one. The birch canoe slid on the smooth planks. Glue the sheet to the dark blue background. It's easy to tell the depth of a well. These days a chicken leg is a rare dish.

Inference Time =  5.237896919250488
Model size =  1763.2071237564087


In [45]:
run("tiny","sample.wav")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



Model name =  tiny

 Harvard List Number One The Birch can use slid on the smooth planks. Do the sheet to the dark blue background. It's easy to tell the depth of a well. These days a chicken leg is a rare dish.

Inference Time =  0.8733940124511719
Model size =  408.8020076751709


# Quantize-arm

In [52]:
quantize_arm("/home/carol/mp/quantize/tiny","/home/carol/mp/quantize/q-tiny-arm")

Creating dynamic quantizer: QOperator (mode: IntegerOps, schema: u8/s8, channel-wise: False)
Quantizing model...
Saving quantized model at: /home/carol/mp/quantize/q-tiny-arm (external data format: False)
Configuration saved in /home/carol/mp/quantize/q-tiny-arm/ort_config.json
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 

quantized  <optimum.onnxruntime.quantization.ORTQuantizer object at 0x7ed4fe3bdc60>


Quantizing model...
Saving quantized model at: /home/carol/mp/quantize/q-tiny-arm (external data format: False)
Configuration saved in /home/carol/mp/quantize/q-tiny-arm/ort_config.json
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49

quantized  <optimum.onnxruntime.quantization.ORTQuantizer object at 0x7ed4fe3bf9d0>


Quantizing model...
Saving quantized model at: /home/carol/mp/quantize/q-tiny-arm (external data format: False)
Configuration saved in /home/carol/mp/quantize/q-tiny-arm/ort_config.json
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49

quantized  <optimum.onnxruntime.quantization.ORTQuantizer object at 0x7ed4e53511b0>
Quantization complete
